In [1]:
%%script bash
# Install en_coref_lg if it hasn't been installed already.
pip show en_coref_lg 2>/dev/null;
if [ ! $? ]; then 
  pip install https://github.com/huggingface/neuralcoref-models/releases/download/en_coref_lg-3.0.0/en_coref_lg-3.0.0.tar.gz;
  pip show en_coref_lg 2>/dev/null;
fi

Name: en-coref-lg
Version: 3.0.0
Summary: Coref-added English multi-task CNN trained on OntoNotes, with GloVe vectors trained on Common Crawl. Assigns word vectors, context-specific token vectors, POS tags, dependency parse, named entities and coreference clusters.
Home-page: https://huggingface.co
Author: HuggingFace Inc. & Explosion AI
Author-email: thomas@huggingface.co
License: CC BY-SA 3.0
Location: /home/dan/.pyenv/versions/3.7.0/lib/python3.7/site-packages
Requires: spacy
Required-by: 


In [2]:
# NLP
import spacy
from spacy import displacy
import en_coref_lg

# Data
import pandas as pd
import csv
import helpers.data as data_helper

# Model
import networkx as nx

# Utils
import time
from graphviz import Source
from tqdm import tqdm
import dill as pickle

In [7]:
! ls ../data

NET_Graphs.pkl	name_entity.csv		       sentences.pkl
document.csv	parsed_sentences.20181205.pkl  trial_graph.pkl
documents.pkl	sentence.csv		       www.glozman.com
download.sh	sentences-coref.pkl
entity_ids.pkl	sentences.20181205.pkl


In [3]:
with open('../data/documents.20181205.pkl', 'rb') as file:
    documents = pickle.load(file)

In [4]:
documents.head()

,document,spacy_tree
document_id,,
bc/cctv/00/cctv_0000@0000@cctv@bc@en@on,"In the summer of 2005 , a picture that people ...","[{'word': 'picture', 'lemma': 'picture', 'NE':..."
bc/cctv/00/cctv_0001@0001@cctv@bc@en@on,What kind of memory ? We respectfully invite y...,"[{'word': 'kind', 'lemma': 'kind', 'NE': '', '..."
bc/cctv/00/cctv_0002@0002@cctv@bc@en@on,Abramov had a car accident in Moscow last nigh...,"[{'word': 'had', 'lemma': 'have', 'NE': '', 'P..."
bc/cctv/00/cctv_0003@0003@cctv@bc@en@on,"Hello , dear viewers . Welcome to Focus Today ...","[{'word': 'Hello', 'lemma': 'hello', 'NE': '',..."
bc/cctv/00/cctv_0004@0004@cctv@bc@en@on,There will be 120 million viewers tuning in to...,"[{'word': 'be', 'lemma': 'be', 'NE': '', 'POS_..."


In [5]:
len(documents)

13109

In [7]:
tree = eval(str(documents.iloc[0].spacy_tree))
print(tree[0]['word'])

picture
